In [27]:
# import tensorflow as tf
# from numpy.random import seed
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array 
from os import listdir
import os
from numpy import asarray
from numpy import save
from tensorflow.keras.applications.inception_v3 import InceptionV3
import urllib.request
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt

In [28]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
mpl.rcParams["figure.figsize"] = (10, 5)
mpl.rcParams["xtick.labelsize"] = 10
mpl.rcParams["ytick.labelsize"] = 10
mpl.rcParams["axes.labelsize"] = 10
mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["legend.fontsize"] = 10
mpl.rcParams["figure.dpi"] = 96

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
training_dir = '/content/drive/MyDrive/data science/02_chest_x_ray/image/train'
validation_dir = '/content/drive/MyDrive/data science/02_chest_x_ray/image/val'
test_dir = '/content/drive/MyDrive/data science/02_chest_x_ray/image/test'

# training_dir = '/Volumes/GoogleDrive/My Drive/data science/02_chest_x_ray/image/train'
# validation_dir = '/Volumes/GoogleDrive/My Drive/data science/02_chest_x_ray/image/val'
# test_dir = '/Volumes/GoogleDrive/My Drive/data science/02_chest_x_ray/image/test'

In [5]:
# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
# except ValueError: # If TPU not found
#   tpu = None

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu) 

In [32]:
# Select appropriate distribution strategy

tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('PU'))

tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.37.201.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.37.201.130:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [31]:
# add a step to convert the pictures to smaller size first
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        training_dir,
        target_size=(300, 300),   
        batch_size = 32,
        class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        batch_size = 32,
        target_size=(300, 300),
        class_mode='binary')

Found 5016 images belonging to 2 classes.
Found 216 images belonging to 2 classes.


In [23]:
# test generator
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        batch_size = 32,
        target_size=(300, 300),
        class_mode='binary')

Found 624 images belonging to 2 classes.


In [ ]:
# this is the same file as just say weights = imageNet
# weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# weights_file = "inception_v3.h5"
# urllib.request.urlretrieve(weights_url, weights_file)
# pre_trained_model = InceptionV3(input_shape=(300, 300, 3), include_top=False, weights=None)
# pre_trained_model.load_weights(weights_file)
# for layer in pre_trained_model.layers:
#     layer.trainable = False

In [33]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

# another call back function overwrite 
# class myCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if(logs.get('accuracy')>0.95):
#             print("\nReached 95% accuracy so cancelling training!")
#             self.model.stop_training = True


In [ ]:
model_dict = {}

Transfer learning with Inception Net

In [ ]:
# transfer learning
# transfer learning with inception network
# with tpu_strategy.scope():
#   pre_trained_model = InceptionV3(input_shape=(300, 300, 3), include_top=False, weights="imagenet")
#   last_layer = pre_trained_model.get_layer('mixed7')
#   # print('last layer output shape: ', last_layer.output_shape)
#   last_output = last_layer.output
#   # Flatten the output layer to 1 dimension
#   x = layers.Flatten()(last_output)
#   # Add a fully connected layer with 1,024 hidden units and ReLU activation
#   x = layers.Dense(512, activation='relu')(x)
#   # Add a dropout rate of 0.2
#   x = layers.Dropout(0.3)(x)
#   # Add a final sigmoid layer for classification
#   x = layers.Dense(1, activation='sigmoid')(x)
#   model = Model(pre_trained_model.input, x)

#   model.compile(optimizer=RMSprop(learning_rate=0.001),
#                 loss='binary_crossentropy',
#                 metrics=['acc'])

# model_name = 'inception_transfer'
# model_dict[model_name] = model

two layers smaller conv net

In [34]:
# two layer small net

with tpu_strategy.scope():
  # train_datagen = ImageDataGenerator(rescale=1/255)
  # train_generator = train_datagen.flow_from_directory(
  #       training_dir,
  #       target_size=(300, 300),   
  #       batch_size = 32,
  #       class_mode='binary')
  # validation_datagen = ImageDataGenerator(rescale=1/255)
  # validation_generator = validation_datagen.flow_from_directory(
  #       validation_dir,
  #       batch_size = 32,
  #       target_size=(300, 300),
  #       class_mode='binary')
  # callback = tf.keras.callbacks.EarlyStopping(
  #   monitor="val_loss",
  #   min_delta=0,
  #   patience=5,
  #   verbose=0,
  #   mode="auto",
  #   baseline=None,
  #   restore_best_weights=True,)

  cnn = tf.keras.Sequential()

  # convolution
  cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(300, 300, 3)))
  cnn.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

  # 2nd Convolution
  cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu"))
  cnn.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

  # Flatten the layer
  cnn.add(tf.keras.layers.Flatten())

  # Fully Connected Layers
  cnn.add(tf.keras.layers.Dense(activation = 'relu', units = 128))
  cnn.add(tf.keras.layers.Dense(activation = 'sigmoid', units = 1))

  # Compile the Neural network
  cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  history = cnn.fit(train_generator,validation_data=validation_generator,epochs=20,verbose=1)


  # model = cnn
  # model_name = 'cnn_model_conv_2L'
  # model.summary()

# model_dict['cnn_model_conv_2L']=model


Epoch 1/20


InternalError: ignored

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_188 (Conv2D)         (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 149, 149, 32)     0         
 2D)                                                             
                                                                 
 conv2d_189 (Conv2D)         (None, 147, 147, 32)      9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 170528)            0         
                                                                 
 dense_4 (Dense)             (None, 128)               2

Conv net with three layers, and drop out

In [ ]:
# from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D


# input_shape = (300, 300, 3)
# model = tf.keras.Sequential()
# # first conv layer
# model.add(Conv2D(32,(3,3),input_shape=input_shape))
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size=(2,2)))

# # second 
# model.add(Conv2D(32,(3,3)))
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size=(2,2)))

# # third
# model.add(Conv2D(64,(3,3)))
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size=(2,2)))

# # flatten
# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation("relu"))
# model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation("sigmoid"))

# model.compile(loss= "binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# model.summary()
# model_name = 'cnn_model_conv_3L_dropout'

# model_dict['cnn_model_conv_3L_dropout'] = model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_96 (Conv2D)          (None, 298, 298, 32)      896       
                                                                 
 activation_94 (Activation)  (None, 298, 298, 32)      0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 149, 149, 32)     0         
 2D)                                                             
                                                                 
 conv2d_97 (Conv2D)          (None, 147, 147, 32)      9248      
                                                                 
 activation_95 (Activation)  (None, 147, 147, 32)      0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                  

maybe two many convs

In [ ]:

# model = tf.keras.models.Sequential([
#     # Note the input shape is the desired size of the image 300x300 with 3 bytes color
#     # This is the first convolution
#     tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     # The second convolution
#     # tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#     # tf.keras.layers.MaxPooling2D(2,2),
#     # # The third convolution
#     # tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     # tf.keras.layers.MaxPooling2D(2,2),
#     # # The fourth convolution
#     # tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     # tf.keras.layers.MaxPooling2D(2,2),
#     # # The fifth convolution
#     # tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     # tf.keras.layers.MaxPooling2D(2,2),
#     # Flatten the results to feed into a DNN
#     tf.keras.layers.Flatten(),
#     # 512 neuron hidden layer
#     tf.keras.layers.Dense(32, activation='relu'),
#     # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 149, 149, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

In [ ]:


# for model_name, model in model_dict.items():
#   print(f'model training started: {model_name}')
  # model.summary()

  # model.save(os.path.join('/content/drive/MyDrive/data science/02_chest_x_ray', model_name))

Epoch 1/20


InternalError: ignored

In [ ]:
# model_name = model_name


In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Model Evaluation

In [ ]:
# model_name = 'cnn_model_conv_2'
# model_name = 'inception_transfer'
# model_name = 'cnn_model_conv_3L_dropout'
model_name = 'cnn_model_conv_2L'

from tensorflow import keras
# model = keras.models.load_model('path/to/location')
# model_name = 'cnn_model_3_layer_20211209'
model = keras.models.load_model(os.path.join('/content/drive/MyDrive/data science/02_chest_x_ray', model_name))

In [ ]:
data_datagen = ImageDataGenerator(rescale=1/255)
data_generator = data_datagen.flow_from_directory(
        # validation_dir,
        test_dir,
        batch_size = 32,
        target_size=(300, 300),
        class_mode='binary', shuffle=False)

Found 624 images belonging to 2 classes.


In [ ]:
# get the predicted value
y_pred_proba_0 = model.predict(data_generator)

In [ ]:
y_pred_proba = list(np.reshape(y_pred_proba_0, -1))
y = list(data_generator.classes)
file_names = data_generator.filenames
data_dict = {'file_name': list(file_names), 
             'y': y,
             "y_pred_proba": y_pred_proba}
df_preds = pd.DataFrame(data_dict)
df_preds['y_pred'] = df_preds['y_pred_proba'].map(lambda x: 1 if x>0.5 else 0)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
precision = precision_score(y_true= df_preds['y'], y_pred = df_preds['y_pred'])
recall = recall_score(y_true= df_preds['y'], y_pred = df_preds['y_pred'])

In [ ]:
print(f"model_name: {model_name}")
print(f"precision: {precision}")
print(f"recall: {recall}")

model_name: cnn_model_conv_2L
precision: 0.7613412228796844
recall: 0.9897435897435898


In [ ]:
# model_name: inception_transfer
# precision: 0.7729083665338645
# recall: 0.9948717948717949

# model_name: cnn_model_conv_3L_dropout
# precision: 0.751953125
# recall: 0.9871794871794872

# model_name: cnn_model_conv_2L
# precision: 0.7613412228796844
# recall: 0.9897435897435898





# inception network: 

In [ ]:
# import pandas as pd

In [ ]:
# compare the actual vs predicted 
# data_dict = {'file_name': list(file_names), 
#              'y': y,
#              "y_pred_proba": preds_1}
# df_preds = pd.DataFrame(data_dict)
# df_preds['y_pred'] = df_preds['y_pred_proba'].map(lambda x: 1 if x>0.5 else 0)
# df_preds['correct_prepdiction_flag'] = (df_preds['y_pred'] == df_preds['y']).astype(int)
# df_preds['correct_prediction_flag'].sum()

In [ ]:
# df_preds.to_csv(os.path.join(project_dir, 'result', f'df_preds_{model_name}.csv'))

In [ ]:
# model.save(os.path.join('/content/drive/MyDrive/data science/02_chest_x_ray', model_name))

In [ ]:
# len(file_names)
# total_images = data_generator.n
# batch_size = 32
# steps = total_images/batch_size 
# round(steps)

# data_generator.reset()
# #iterations to cover all data, so if batch is 5, it will take total_images/5  iteration 
# x , y = [] , []
# for i in range(round(steps)):
#     a , b = data_generator.next()
#     y.extend(b)  